---
title: "Online Feature Serving"
date: 2021-04-25
type: technical_note
draft: false
---

## Connect to the Feature Store

In [1]:
import hsfs

connection = hsfs.connection()
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


## Check serving keys

In [2]:
td_meta = fs.get_training_dataset("card_fraud_dataset_tf", 1)

To identify which primary key names are used for this training dataset query use serving_keys method

In [3]:
#`init_prepared_statement` method is needed to get serving_keys in case `get_serving_vector` has not beed called yet. This is not necessary for `get_serving_vector` method itself
td_meta.init_prepared_statement() 
td_meta.serving_keys

{'cc_num'}

## Deploy fraud detector model

In [4]:
from hops import model, serving
from hops.model import Metric

In [5]:
MODEL_NAME="card_fraud_detector"
SERVING_NAME="cardfrauddetector"
TOPIC_NAME = "credit_card_inference"

best_model = model.get_best_model(MODEL_NAME, "accuracy", Metric.MAX)

print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: card_fraud_detector
Model version: 1
{'accuracy': '0.9509761333465576'}


In [6]:
model_path="/Models/" + best_model['name']
transformer_path =  "/Projects/card_fraud/Jupyter/recent_card_activity_transformer.py"

# Create model serving
response = serving.create_or_update(SERVING_NAME, model_path, model_version=best_model['version'], artifact_version="CREATE",
                                    transformer=transformer_path, kfserving=True, topic_name=TOPIC_NAME)

Inferring model server from artifact files: TENSORFLOW_SERVING
Creating serving cardfrauddetector for artifact /Projects/card_fraud//Models/card_fraud_detector ...
Serving cardfrauddetector successfully created


In [7]:
# Start model serving
serving.start(SERVING_NAME)

Starting serving with name: cardfrauddetector...
Serving with name: cardfrauddetector successfully started


In [8]:
# Wait for it
import time
while serving.get_status(SERVING_NAME) != "Running":
    time.sleep(5)
time.sleep(5)

## Get credit card numbers

In [9]:
fraud_labels_fg = fs.get_feature_group("fraud_labels", version=1)
cc_num = fraud_labels_fg.select("cc_num")
card_numbers = cc_num.read()

Lazily executing query: SELECT `fg0`.`cc_num`
FROM `card_fraud_featurestore`.`fraud_labels_1` `fg0`


In [10]:
card_numbers_inputs = card_numbers.sample(n=len(card_numbers)*20, replace=True)

## Make inference requests

In [14]:
i=0
for cc_num in card_numbers_inputs['fg0.cc_num']:
    data = { "signature_name": "serving_default", "instances": [{'cc_num': int(cc_num)}] }
    response = serving.make_inference_request(SERVING_NAME, data)
    if i % 100 == 0:
        print(response)
    i+=1

{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[1.0]]}
{'predictions': [[1.0]]}
{'predictions': [[8.01005353e-06]]}
{'predictions': [[3.81734644e-08]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[1.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[1.71964825e-08]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.0]]}
{'predictions': [[0.993911743]]}
{'predictions': [[0.0]]}
{'predict